In [1]:
import os,sys,string, time
import ROOT
from math import *
from ROOT import gPad, TTree, TObject, TFile, gDirectory, TH1D, TH2D, TH3D, TCanvas, gROOT, TGaxis, gStyle, TColor, TLegend, THStack, TChain, TLatex, TText, TCollection, kRed, kBlue
from array import array
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from root_pandas import read_root
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions

print('Success')

Welcome to JupyROOT 6.24/06
Success


## Reading in single file

In [4]:
def Single_file_POT(location): #Returns the cumulative POT of the sample at the "location"
    file = uproot3.open(location)['nuselection/SubRun']
    print("Number of branches is " + str(len(file.keys()))) 
    print("Number of subruns is " + str(file.numentries) + "\n")
    file_evs = uproot3.open(location)['nuselection/NeutrinoSelectionFilter']
    print("Number of events is " + str(file_evs.numentries))
    POT = Functions.POT_counter(file)
    print("Total POT in file is " + str(POT))
    return POT

def Single_file_POT_scaling(POT, Run, file_type): #Do not use for EXT
    print("Calculating scaling for " + Run + " " + file_type)
    if Run == "run1":
        Data_POT = Constants.Run1_POT
        further_scaling = {"signal":Constants.NuMI_KDAR_scaling_run1,
                           "overlay":1.0,
                           "dirt":Constants.DIRT_run1_scaling}
    if Run == "run3":
        Data_POT = Constants.Run3_POT
        further_scaling = {"signal":Constants.NuMI_KDAR_scaling_run3,
                           "overlay":1.0,
                           "dirt":Constants.DIRT_run3_scaling}
    SF = (Data_POT/POT)*further_scaling[file_type]
    print("Scale factor is " + str(SF))
    return SF
    
# loc = '../NuMI_signal/KDAR_dump/sfnues/sfnues_KDAR_dump_10_Umu4_majorana_numi_FHC.root'
loc = '../NuMI_signal/KDAR_dump/sfnues/DetVars/150_CV_run1.root'
file_type = "signal" #"signal", "overlay" or "dirt"
Run="run1"
POT = Single_file_POT(loc)

Single_file_POT_scaling(POT, Run, file_type)


Number of branches is 3
Number of subruns is 4642

Number of events is 35201
Total POT in file is 4.453505e+26
Calculating scaling for run1 signal
Scale factor is 3.592676027097128e-06


3.592676027097128e-06

In [5]:
loc = '../NuMI_signal/KDAR_dump/sfnues/sfnues_KDAR_dump_150_Umu4_majorana_numi_FHC.root'
file_type = "signal" #"signal", "overlay" or "dirt"
Run="run1"
POT = Single_file_POT(loc)

Single_file_POT_scaling(POT, Run, file_type)


Number of branches is 3
Number of subruns is 4667

Number of events is 35555
Total POT in file is 4.8456162e+26
Calculating scaling for run1 signal
Scale factor is 3.301953636715338e-06


3.301953636715338e-06

In [7]:
Event_ratio = 35201/35555

POT_ratio = 4.453505e+26/4.8456162e+26

ev_per_POT_1 = 35201/4.453505e+26
ev_per_POT_2 = 35555/4.8456162e+26

SF = ev_per_POT_1/ev_per_POT_2

print("Event ratio is " + str(Event_ratio))
print("POT ratio is " + str(POT_ratio))
print("Scale factor should be " + str(SF))

Event ratio is 0.990043594431163
POT ratio is 0.9190791874932233
Scale factor should be 1.077212505629133


## Reading in all files

In [7]:
#This reads in the MC overlay, MC dirt overlay, EXT, data and signal samples for NuMI run1 or run3
#HNL_masses = Constants.HNL_mass_samples #in MeV

Run = "run1" #so far either "run1" or "run3"

Load_pi0_samples = True

FLATTEN = False

root_dir = 'nuselection'
POT_tree = 'SubRun' #Branch for POT
MC_samples_dir = '../NuMI_MC/'
data_samples_dir = '../NuMI_data/'
signal_samples_dir = '../NuMI_signal/KDAR_dump/sfnues/'

loc_overlay_run1 = MC_samples_dir+'SLIMMED_neutrinoselection_filt_run1_overlay.root'#NuMI Run1 MC WITHOUT systematics weights
loc_dirt_run1 = MC_samples_dir+'neutrinoselection_filt_run1_dirt_overlay.root'
loc_EXT_run1 = data_samples_dir+'neutrinoselection_filt_run1_beamoff.root'
loc_beamgood_run1 = data_samples_dir+'neutrinoselection_filt_run1_beamon_beamgood.root'

loc_overlay_run3 = MC_samples_dir+'SLIMMED_neutrinoselection_filt_run3_overlay.root' #NuMI Run3 MC WITHOUT systematics weights
loc_dirt_run3 = MC_samples_dir+'neutrinoselection_filt_run3_dirt_overlay.root'
loc_EXT_run3 = data_samples_dir+'neutrinoselection_filt_run3_beamoff.root'
loc_beamgood_run3 = data_samples_dir+'neutrinoselection_filt_run3_beamon_beamgood.root'

print("Opening Run1 samples with uproot")
NuMI_MC_overlay_run1 = uproot3.open(loc_overlay_run1)[root_dir+'/'+POT_tree]
NuMI_MC_dirt_run1 = uproot3.open(loc_dirt_run1)[root_dir+'/'+POT_tree]
NuMI_EXT_run1 = uproot3.open(loc_EXT_run1)[root_dir+'/'+POT_tree]
NuMI_beamgood_run1 = uproot3.open(loc_beamgood_run1)[root_dir+'/'+POT_tree]

signal_samples_dict_run1 = {}
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        Signal_run1  = uproot3.open(signal_samples_dir+f'pi0/sfnues_pi0_KDAR_dump_{HNL_mass}_Umu4_majorana_numi_FHC.root')[root_dir+'/'+POT_tree]
        signal_samples_dict_run1[HNL_mass] = Signal_run1
else:
    for HNL_mass in Constants.HNL_mass_samples:
        Signal_run1  = uproot3.open(signal_samples_dir+f'sfnues_KDAR_dump_{HNL_mass}_Umu4_majorana_numi_FHC.root')[root_dir+'/'+POT_tree]
        signal_samples_dict_run1[HNL_mass] = Signal_run1
    
print("Opening Run3 samples with uproot")
NuMI_MC_overlay_run3 = uproot3.open(loc_overlay_run3)[root_dir+'/'+POT_tree]
NuMI_MC_dirt_run3 = uproot3.open(loc_dirt_run3)[root_dir+'/'+POT_tree]
NuMI_EXT_run3 = uproot3.open(loc_EXT_run3)[root_dir+'/'+POT_tree]
NuMI_beamgood_run3 = uproot3.open(loc_beamgood_run3)[root_dir+'/'+POT_tree]

signal_samples_dict_run3 = {}
for HNL_mass in Constants.HNL_mass_samples:
    Signal_run3  = uproot3.open(signal_samples_dir+f'sfnues_KDAR_dump_{HNL_mass}_Umu4_majorana_numi_RHC.root')[root_dir+'/'+POT_tree]
    signal_samples_dict_run3[HNL_mass] = Signal_run3

print("Opened files" + "\n")
print("----RUN1----"+ "\n")
print("----MC OVERLAY BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_overlay_run1.keys()))) 
print("Number of subruns is " + str(NuMI_MC_overlay_run1.numentries))
print("----MC DIRT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_dirt_run1.keys()))) 
print("Number of subruns is " + str(NuMI_MC_dirt_run1.numentries))
print("----EXT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_EXT_run1.keys()))) 
print("Number of subruns is " + str(NuMI_EXT_run1.numentries))
print("----DATA----")
print("Number of branches is " + str(len(NuMI_beamgood_run1.keys()))) 
print("Number of subruns is " + str(NuMI_beamgood_run1.numentries))
print("----SIGNAL----")
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        print(f"Number of branches in pi0 {HNL_mass}MeV is " + str(len(signal_samples_dict_run1[HNL_mass].keys()))) 
        print(f"Number of subruns in pi0 {HNL_mass}MeV is " + str(signal_samples_dict_run1[HNL_mass].numentries))
else:
    for HNL_mass in Constants.HNL_mass_samples:
        print(f"Number of branches in {HNL_mass}MeV is " + str(len(signal_samples_dict_run1[HNL_mass].keys()))) 
        print(f"Number of subruns in {HNL_mass}MeV is " + str(signal_samples_dict_run1[HNL_mass].numentries))

print()

print("----RUN3----"+ "\n")
print("----MC OVERLAY BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_overlay_run3.keys()))) 
print("Number of subruns is " + str(NuMI_MC_overlay_run3.numentries))
print("----MC DIRT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_dirt_run3.keys()))) 
print("Number of subruns is " + str(NuMI_MC_dirt_run3.numentries))
print("----EXT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_EXT_run3.keys()))) 
print("Number of subruns is " + str(NuMI_EXT_run3.numentries))
print("----DATA----")
print("Number of branches is " + str(len(NuMI_beamgood_run3.keys()))) 
print("Number of subruns is " + str(NuMI_beamgood_run3.numentries))
print("----SIGNAL----")
for HNL_mass in Constants.HNL_mass_samples:
    print(f"Number of branches in {HNL_mass}MeV is " + str(len(signal_samples_dict_run3[HNL_mass].keys()))) 
    print(f"Number of subruns in {HNL_mass}MeV is " + str(signal_samples_dict_run3[HNL_mass].numentries))

print()

print("Done!")

Opening Run1 samples with uproot
Opening Run3 samples with uproot
Opened files

----RUN1----

----MC OVERLAY BACKGROUND----
Number of branches is 3
Number of subruns is 59215
----MC DIRT BACKGROUND----
Number of branches is 3
Number of subruns is 13085
----EXT BACKGROUND----
Number of branches is 2
Number of subruns is 158607
----DATA----
Number of branches is 2
Number of subruns is 82200
----SIGNAL----
Number of branches in pi0 150MeV is 3
Number of subruns in pi0 150MeV is 4954

----RUN3----

----MC OVERLAY BACKGROUND----
Number of branches is 3
Number of subruns is 80063
----MC DIRT BACKGROUND----
Number of branches is 3
Number of subruns is 38197
----EXT BACKGROUND----
Number of branches is 2
Number of subruns is 366514
----DATA----
Number of branches is 2
Number of subruns is 286792
----SIGNAL----
Number of branches in 20MeV is 3
Number of subruns in 20MeV is 4970
Number of branches in 50MeV is 3
Number of subruns in 50MeV is 5051
Number of branches in 100MeV is 3
Number of subrun

# Checking POT normalisation

In [8]:
def POT_counter_old(df): #This takes dataframe, which I probably don't need to load anymore
    Total_POT = 0
    for i in range(len(df['pot'])):
        Total_POT += df['pot'][i]
    return Total_POT

def POT_counter(file): #Takes uproot file
    Total_POT = file["pot"].array().sum()
    return Total_POT

signal_POT_dict_run1 = {}

#-----Run1-----#
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        signal_POT_run1 = POT_counter(signal_samples_dict_run1[HNL_mass])
        signal_POT_dict_run1[HNL_mass] = signal_POT_run1
else:
    for HNL_mass in Constants.HNL_mass_samples:
        signal_POT_run1 = POT_counter(signal_samples_dict_run1[HNL_mass])
        signal_POT_dict_run1[HNL_mass] = signal_POT_run1

overlay_POT_run1 = POT_counter(NuMI_MC_overlay_run1)
dirt_POT_run1 = POT_counter(NuMI_MC_dirt_run1)
#beamgood_POT = POT_counter(df_beamgood_run1) #There is no 'pot' branch for beamgood
OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run1 = 9199232.74 #Taken from the NuMI samples page

#-----Run3-----#
signal_POT_dict_run3 = {}

for HNL_mass in Constants.HNL_mass_samples:
    signal_POT_run3 = POT_counter(signal_samples_dict_run3[HNL_mass])
    signal_POT_dict_run3[HNL_mass] = signal_POT_run3
    

overlay_POT_run3 = POT_counter(NuMI_MC_overlay_run3)
dirt_POT_run3 = POT_counter(NuMI_MC_dirt_run3)
#beamgood_POT = POT_counter(df_beamgood_run1) #There is no 'pot' branch for beamgood
OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run3 = 32878305.25

#Signal
print("----RUN1----"+ "\n")
print('Total signal POTs are: ')
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        print(f"{HNL_mass}MeV : "  + str(signal_POT_dict_run1[HNL_mass]))
else:
    for HNL_mass in Constants.HNL_mass_samples:
        print(f"{HNL_mass}MeV : "  + str(signal_POT_dict_run1[HNL_mass]))
print('-------------------')
#Dirt
print('Total dirt POT is ' + str(dirt_POT_run1))
print('-------------------')
#Overlay
print('Total overlay POT is ' + str(overlay_POT_run1))
print('-------------------'+ "\n")
print("----RUN3----"+ "\n")
print('Total signal POTs are: ')
for HNL_mass in Constants.HNL_mass_samples:
    print(f"{HNL_mass}MeV : "  + str(signal_POT_dict_run3[HNL_mass]))
print('-------------------')
#Dirt
print('Total dirt POT is ' + str(dirt_POT_run3))
print('-------------------')
#Overlay
print('Total overlay POT is ' + str(overlay_POT_run3))
print('-------------------')

----RUN1----

Total signal POTs are: 
150MeV : 3.8463033e+26
-------------------
Total dirt POT is 1.6739124e+21
-------------------
Total overlay POT is 2.3365224e+21
-------------------

----RUN3----

Total signal POTs are: 
20MeV : 9.7409455e+32
50MeV : 3.156666e+30
100MeV : 3.1090238e+28
150MeV : 6.557156e+26
180MeV : 6.5540295e+25
200MeV : 2.3616463e+25
-------------------
Total dirt POT is 1.0322625e+21
-------------------
Total overlay POT is 1.9893618e+21
-------------------


In [11]:
Run1_POT = 2e20 #NEEDS TO BE CHECKED, just taken from the NuMI samples page
Run3_POT = 5.0e20 #NEEDS TO BE CHECKED, just taken from the NuMI samples page
OnBeam_EA9CNT_wcut_run1 = 5268051.0 #"Triggers" taken from the NuMI samples page
OnBeam_EA9CNT_wcut_run3 = 10363728.0 #"Triggers" taken from the NuMI samples page
BeamOff_scaling_for_nus = 0.98 #This Factor described by Owen: 
# An additional scaling
# factor of 0.98 is applied to the Beam-off sample to take
# into account that 2% of all NuMI Beam-on events are
# expected to contain a neutrino interaction
DIRT_run1_scaling = 0.75 #NOT SURE where this comes from, apparently it is standard procedure for NuMI DIRT
DIRT_run3_scaling = 0.35 #NOT SURE where this comes from, apparently it is standard procedure for NuMI DIRT
NuMI_KDAR_scaling_run1 = 8.0 #This comes from the discrepancy between numu flux from KDAR dump between Geant4 and MiniBooNE measurement. Taken from Owen's thesis
NuMI_KDAR_scaling_run3 = 8.6

run1_POT_scaling_dict = {}
run3_POT_scaling_dict = {}

#Calculation of POT scaling factors
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        SF_signal_run1 = (Run1_POT/signal_POT_dict_run1[HNL_mass])*NuMI_KDAR_scaling_run1
        run1_POT_scaling_dict[HNL_mass] = SF_signal_run1
else:
    for HNL_mass in Constants.HNL_mass_samples:
        SF_signal_run1 = (Run1_POT/signal_POT_dict_run1[HNL_mass])*NuMI_KDAR_scaling_run1
        run1_POT_scaling_dict[HNL_mass] = SF_signal_run1
        
SF_overlay_run1 = Run1_POT/overlay_POT_run1
SF_dirt_run1 = (Run1_POT/dirt_POT_run1)*DIRT_run1_scaling
SF_EXT_run1 = (OnBeam_EA9CNT_wcut_run1/OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run1)*BeamOff_scaling_for_nus
    
for HNL_mass in Constants.HNL_mass_samples:
    SF_signal_run3 = (Run1_POT/signal_POT_dict_run3[HNL_mass])*NuMI_KDAR_scaling_run3
    run3_POT_scaling_dict[HNL_mass] = SF_signal_run3
    
SF_overlay_run3 = Run3_POT/overlay_POT_run3
SF_dirt_run3 = (Run3_POT/dirt_POT_run3)*DIRT_run3_scaling
SF_EXT_run3 = (OnBeam_EA9CNT_wcut_run3/OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run3)*BeamOff_scaling_for_nus


print("The following factors can be applied to the full samples, i.e they are not event-dependent")
print()
print('For Run1 the scale factors are: ')
print('Overlay: ' + str(SF_overlay_run1))
print('Dirt: ' + str(SF_dirt_run1))
print('EXT: ' + str(SF_EXT_run1))
print('Signal: ')
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        print(f"{HNL_mass}MeV " + str(run1_POT_scaling_dict[HNL_mass]))
else:
    for HNL_mass in Constants.HNL_mass_samples:
        print(f"{HNL_mass}MeV " + str(run1_POT_scaling_dict[HNL_mass]))

print()
print('For Run3 the scale factors are: ')
print('Overlay: ' + str(SF_overlay_run3))
print('Dirt: ' + str(SF_dirt_run3))
print('EXT: ' + str(SF_EXT_run3))
print('Signal: ')
for HNL_mass in Constants.HNL_mass_samples:
    print(f"{HNL_mass}MeV " + str(run3_POT_scaling_dict[HNL_mass]))
    

The following factors can be applied to the full samples, i.e they are not event-dependent

For Run1 the scale factors are: 
Overlay: 0.08559729716258113
Dirt: 0.08961042376561343
EXT: 0.5612087579382191
Signal: 
150MeV 4.159838350914407e-06

For Run3 the scale factors are: 
Overlay: 0.2513368817255014
Dirt: 0.16953052634982632
EXT: 0.3089104916683624
Signal: 
20MeV 1.7657423469953374e-12
50MeV 5.448786613287057e-10
100MeV 5.5322831399190343e-08
150MeV 2.623088564712337e-06
180MeV 2.624339736997261e-05
200MeV 7.283055184784342e-05
